In [48]:
from collections import namedtuple
import math

import numpy as np
import z3
from z3 import *

def car(real, imag):
    return complex(real,imag)

def pol(rho, phi):
    return (rho, phi)

def car2pol(car):
    x, y = car.real, car.imag
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    phi = phi / 2 / math.pi * 360
    return(rho, phi)

def pol2car(rho, phi):
    phi = phi / 360 * 2 * math.pi
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return complex(x, y)

In [49]:
def checkcar(var):
    if isinstance(var, tuple):
        print('converting {var} into carthesian to calculate with')
        return pol2car(*var)
    return var

def checkpol(var):
    if isinstance(var, complex):
        print('converting {var} into polar to calculate with')
        return car2pol(var)
    return var

def parallel(x1,x2):
    return (x1*x2) / (x1 + x2)

def current(voltage, resistance):
    voltage = checkcar(voltage)
    cur = voltage / resistance
    print('current in polar notation', car2pol(cur))
    return cur

def voltage(current, resistance):
    current = checkcar(current)
    vol = current * resistance
    print('voltage in polar notation', car2pol(vol))
    return vol

def deg2rad(deg):
    return deg / 360 * 2 * math.pi

def rad2deg(rad):
    return rad / 2 / math.pi * 360


def Q(voltage, current):
    voltage = checkpol(voltage)
    current = checkpol(current)
    print(f'  {voltage[0]} * {current[0]} * sin({voltage[1]} - {current[1]}) WARNING, make sure to multiply with -1 if the current enters at -')
    return voltage[0] * current[0] * math.sin(deg2rad(voltage[1] - current[1]))
    
def P(voltage, current):
    voltage = checkpol(voltage)
    current = checkpol(current)
    print(f'  {voltage[0]} * {current[0]} * cos({voltage[1]} - {current[1]}) WARNING, make sure to multiply with -1 if the current enters at -')
    return voltage[0] * current[0] * math.cos(deg2rad(voltage[1] - current[1]))

def S(P, Q):
    return math.sqrt(P**2 + Q**2)



In [50]:
def capacitor_resistance(capacity, frequency=50):
    return -1 / (2*math.pi * frequency) / capacity

def capacitor_capacity(resistance, frequency=50):
    return 1 / (resistance * 2*math.pi * frequency)
    
def inductor_resistance(capacity, frequency=50):
    return (2*math.pi * frequency) * capacity

def inductor_capacity(resistance, frequency=50):
    return resistance / (2*math.pi*frequency)

In [51]:
def car2complexexpr(cart):
    return ComplexExpr(cart.real, cart.imag)

def pol2complexexpr(size, angle):
    return car2complexexpr(pol2car(size, angle))

def _to_complex(a):
    if isinstance(a, ComplexExpr):
        return a
    else:
        return ComplexExpr(a, RealVal(0))

def _is_zero(a):
    return (isinstance(a, int) and a == 0) or (is_rational_value(a) and a.numerator_as_long() == 0)

class ComplexExpr:
    def __init__(self, r, i):
        self.r = r
        self.i = i

    def __add__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r + other.r, self.i + other.i)

    def __radd__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r + self.r, other.i + self.i)

    def __sub__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r - other.r, self.i - other.i)

    def __rsub__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r - self.r, other.i - self.i)

    def __mul__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r*other.r - self.i*other.i, self.r*other.i + self.i*other.r)

    def __mul__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r*self.r - other.i*self.i, other.i*self.r + other.r*self.i)

    def inv(self):
        den = self.r*self.r + self.i*self.i
        return ComplexExpr(self.r/den, -self.i/den)

    def __div__(self, other):
        inv_other = _to_complex(other).inv()
        return self.__mul__(inv_other)

    def __rdiv__(self, other):
        other = _to_complex(other)
        return self.inv().__mul__(other)

    def __eq__(self, other):
        other = _to_complex(other)
        return And(self.r == other.r, self.i == other.i)

    def __neq__(self, other):
        return Not(self.__eq__(other))

    def simplify(self):
        return ComplexExpr(simplify(self.r), simplify(self.i))

    def repr_i(self):
        if is_rational_value(self.i):
            return "%s*I" % self.i
        else:
            return "(%s)*I" % str(self.i)

    def __repr__(self):
        if _is_zero(self.i):
            return str(self.r)
        elif _is_zero(self.r):
            return self.repr_i()
        else:
            return "%s + %s" % (self.r, self.repr_i())

def Complex(a):
    return ComplexExpr(Real('%s.r' % a), Real('%s.i' % a))
# I = ComplexExpr(RealVal(0), RealVal(1))

def evaluate_cexpr(m, e):
    return ComplexExpr(m[e.r], m[e.i])

def get_ans(m, var):
    r = evaluate_cexpr(m, var).r
    i = evaluate_cexpr(m, var).i
    real = float(r.numerator_as_long())/float(r.denominator_as_long())
    imag = float(i.numerator_as_long())/float(i.denominator_as_long())
    return complex(real, imag)

In [52]:
# template for AC nets
I1 = Complex("x")
I2 = Complex("y")
I3 = Complex("z")
s = Tactic('qfnra-nlsat').solver()





# s.add(ComplexExpr(cur.real, cur.imag) - I1 * ComplexExpr(8,4) - I2*ComplexExpr(0,-6) == 0)
# s.add(I2 * ComplexExpr(0,-6) - ComplexExpr(cur2.real, cur2.imag) == 0)
# s.add(I1 + I3 - I2 == 0)

s.add(100-I1*ComplexExpr(0,6.28)-I3*ComplexExpr(0,-6)==0)
s.add(I3*ComplexExpr(0,-6)+I2*10 == 0)
s.add(I1 + I2 -I3 == 0)
s.check()
m = s.model()

for var in [I1,I2, I3]:
    print(car2pol(get_ans(m,var)))
    

(30.864750979992415, -35.21359368159654)
(15.879783371533687, 85.75016285047698)
(26.466305619222812, -4.249837149523023)


In [53]:
# template for DC nets
I1 = Real('x')
I2 = Real('y')
I3 = Real('z')
I4 = Real('r')
I5 = Real('t')

s = Tactic('qfnra-nlsat').solver()
s.add(-10*I1-100-5*I2==0)
s.add(5*I2 + 100 -20*I4 == 0)
s.add(20*I4 + 40 == 0)

s.add(I1 - I2 - I3 == 0)
s.add(I3 - I4 - I5 == 0)
    
s.check()
m = s.model()

m.eval(I1), m.eval(I2), m.eval(I3),m.eval(I4),m.eval(I5)

(4, -28, 32, -2, 34)

In [54]:
# template for DC nets
R = Real('x')
I2 = Real('y')
I3 = Real('z')
I4 = Real('r')
I5 = Real('t')

s = Tactic('qfnra-nlsat').solver()
s.add(40 == (60 * (16+R)) / (60 + (16+R)))
    
s.check()
m = s.model()

m.eval(R)

-76

In [55]:
Power = namedtuple('Power', 'Watt VAR')

def power(current, resistance):
    if isinstance(resistance, complex):
        return Power(power(current, resistance.real)[0], 
                     power(current, resistance.imag)[0])
    else:
        return Power(car2pol(current ** 2 * resistance)[0],0)


def trans(primaryvoltage, primaryresistance, secondaryresistance, ratio):
    # bring everything to left side
    primaryvoltage = checkcar(primaryvoltage)
    res = primaryresistance + ratio ** 2 * secondaryresistance
    Iprim = current(primaryvoltage, res)
    Isec = Iprim * ratio
    
    Uprim = checkcar(primaryvoltage) - primaryresistance * Iprim
    Usec  = Uprim / ratio
    print(f'Transformer: \nIp {car2pol(Iprim)}\nIs {car2pol(Isec)}\nUp {car2pol(Uprim)}\nUs {car2pol(Usec)}')
    print(f'Powers on primary side\nPsource {P(primaryvoltage, Iprim)}, Qsource{Q(primaryvoltage, Iprim)}\n')
    print(f'Resistances on primary side\nP {power(Iprim, primaryresistance)}\n')
    print(f'Resistances on secondary side\nP {power(Isec, secondaryresistance)}\n')
    print(f'Efficiency transformer\n{power(Isec, secondaryresistance).Watt / P(primaryvoltage, Iprim)}\n')

trans((100,0), 2, (100-200j), 0.1)

converting {var} into carthesian to calculate with
current in polar notation (27.735009811261456, 33.6900675259798)
Transformer: 
Ip (27.735009811261456, 33.6900675259798)
Is (2.7735009811261455, 33.6900675259798)
Up (62.017367294604234, -29.744881296942225)
Us (620.1736729460423, -29.74488129694223)
converting {var} into polar to calculate with
converting {var} into polar to calculate with
  100.0 * 27.735009811261456 * cos(0.0 - 33.6900675259798) WARNING, make sure to multiply with -1 if the current enters at -
converting {var} into polar to calculate with
converting {var} into polar to calculate with
  100.0 * 27.735009811261456 * sin(0.0 - 33.6900675259798) WARNING, make sure to multiply with -1 if the current enters at -
Powers on primary side
Psource 2307.6923076923076, Qsource-1538.4615384615388

Resistances on primary side
P Power(Watt=1538.4615384615383, VAR=0)

Resistances on secondary side
P Power(Watt=769.2307692307692, VAR=1538.4615384615383)

converting {var} into polar t

### Asynchronous machines

In [56]:
def nullast(P,U,I):
    S = U * I # schijnbaar vermogen
    Q = (S**2 - P**2)**0.5 # blind vermogen
    # nullasten staan parallel aan de bron dus P = U**2 / R gebruiken
    Rij = U**2 / P  
    Xij = U**2 / Q
    phi = rad2deg(math.acos((P / (U * I)))) # from formula P = U * I * cos(phi)
    print(
        'Nullast:\n'
        f'S = {S}\n'
        f'Q = {Q}\n'
        f'phi = {phi}\n'
        f'Rij = {Rij}\n'
        f'Xij = {Xij}\n')
    return Rij, Xij


def kortsluit(P,U,I):
    S = U * I # schijnbaar vermogen
    Q = (S**2 - P**2)**0.5 # blind vermogen
    # nullasten staan in serie met bron dus P = I**2 * R gebruiken
    Rij = P / I**2
    Xij = Q / I**2
    phi = rad2deg(math.acos((P / (U * I)))) # from formula P = U * I * cos(phi)
    print(
        'Kortsluit:\n'
        f'S = {S}\n'
        f'Q = {Q}\n'
        f'phi = {phi}\n'
        f'Rij = {Rij}\n'
        f'Xij = {Xij}\n')
    return Rij, Xij



In [58]:
def slip(fnet, frotor):
    return (fnet - frotor) / fnet
slip(50,0), slip(50,50)

(1.0, 0.0)

In [59]:
def Pas(I, R2, s):
    # asvermogen bepalen
    return 3 * I**2 * R2 * (1-s) / s

def R2(Rk, ohmmeasured):
    # vervangingsweerstand as berekenen bij bekend zijn van Rk vanuit kortsluitproef
    return Rk - (ohmmeasured / 2)

def Ras(R2, slip):
    # vervangingsweerstand as berekenen inclusief slip
    return R2 * (1-slip) / slip

In [60]:
nullast(180/3, 230, 0.4)

Nullast:
S = 92.0
Q = 69.74238309665078
phi = 49.29429316895856
Rij = 881.6666666666666
Xij = 758.5057701095448



(881.6666666666666, 758.5057701095448)

In [62]:
kortsluit(1500/3, 0.05*230, 0.6*130.4)

Kortsluit:
S = 899.76
Q = 748.0428180258133
phi = 56.24079944551819
Rij = 0.08167935898561818
Xij = 0.12219931574028771



(0.08167935898561818, 0.12219931574028771)

In [ ]:
def koppel(P, freq):
    # P = w T
    return p / (48.94 * 2 * math.pi)

In [64]:
def slip2freq(s):
    # slip = (fnet - frot) / fnet
    return -(s * 50 - 50)
slip2freq(0.0212)

48.94

In [66]:
def asynchrone(voltage, Xk, Rk, R2, slip):
    # gegeven vervangingsschema en een slip bepaal stroom, vermogens en rendement
    if isinstance(voltage, int):
        voltage = (voltage, 0)
    reactance = complex(Rk + Ras(R2, slip), Xk)
    cur = checkpol(current(voltage, reactance))
    print('total reactance', reactance)
    print('current', cur)
    print('vermogen', 3* P(voltage, cur))
    print('blindvermogen', 3*Q(voltage, cur))
    print('asvermogen', Pas(cur[0], R2, slip))
    print('rendement bij slip > 0 (anders 1/x doen)', Pas(cur[0], R2, slip) / (3 * P(voltage, cur)))
    return Pas(cur[0], R2, slip)
    
    
asynchrone(230, 0.2, 0.206, 0.156, -0.033333)

converting {var} into carthesian to calculate with
current in polar notation (49.62924367654542, -177.52658296177944)
converting {var} into polar to calculate with
total reactance (-4.630046800468004+0.2j)
current (49.62924367654542, -177.52658296177944)
  230 * 49.62924367654542 * cos(0 - -177.52658296177944) WARNING, make sure to multiply with -1 if the current enters at -
vermogen -34212.274606952094
  230 * 49.62924367654542 * sin(0 - -177.52658296177944) WARNING, make sure to multiply with -1 if the current enters at -
blindvermogen 1477.8370967435362
asvermogen -35734.44681659795
  230 * 49.62924367654542 * cos(0 - -177.52658296177944) WARNING, make sure to multiply with -1 if the current enters at -
rendement bij slip > 0 (anders 1/x doen) 1.0444919908755947


-35734.44681659795

### Synchnonous machines

In [116]:
def X1(S1f, Unom, I2nullast, I1kortsluit, I2kortsluit):
    lam = Unom / I2nullast
    print(f'Bij een I2 van {I2nullast}A wordt de nominale fasespanning gehaald van {Unom}V')
    I1nullast = I1kortsluit / I2kortsluit * I2nullast
    print(f'Bij een I2 van {I2kortsluit}A treedt een I1 op van {I1kortsluit}A, dus bij I2 van {I2nullast}A is I1 {I1nullast}A')
    X1 = Unom / I1nullast
    print(f'X1 is daarom {Unom} / {I1nullast} = {X1}ohm')
    return X1
#17.1
x1 = X1(56679/3, 230, 2.3, Imax*0.8, 9.2)

Bij een I2 van 2.3A wordt de nominale fasespanning gehaald van 230V
Bij een I2 van 9.2A treedt een I1 op van 65.71478260869566A, dus bij I2 van 2.3A is I1 16.428695652173914A
X1 is daarom 230 / 16.428695652173914 = 13.999894140687026ohm


In [196]:
def U12(Unet, I1, X1, l = None):
    UX1 = voltage(I1, complex(0,X1))
    print(f'Voltage over X1 {car2pol(UX1)}')
    u12 = car2pol(pol2car(*Unet) - UX1)
    print(f'Unet - UX1 = U12 --> {Unet} - {car2pol(UX1)} = {u12}')
    print(f'lasthoek is {u12[1] - Unet[1]}')
    print(f'Pas is {3 * -P(u12, I1)}')
    QX1 = 3 * I1[0]**2 * X1
    Qas = 3 * Q(u12, I1)
    print(f'Qas is {Qas}')
    print(f'QX1 is {QX1}')
    print(f'Pnet is {-3 * P(Unet, I1)}')
    print(f'Qnet is {-3 * Q(Unet, I1)}')
    machineQ = QX1 + Qas
    behavior = 'inductive' if machineQ > 0 else 'capacitive'
    print(f'machine is operating {machineQ}, therefore {behavior}')
    
    
    if l:
        I2 = u12[0] / l
        print(f'I2 is {I2}A')
    return u12

In [197]:
unet = (230,0)
z = complex(3,2)
inet = car2pol(pol2car(*unet) / z)
i1 = inet[0], inet[1] - 180

In [203]:
U12(unet, (50,20), 9.86,57.5)

converting {var} into carthesian to calculate with
voltage in polar notation (492.99999999999994, 110.00000000000001)
Voltage over X1 (492.99999999999994, 110.00000000000001)
Unet - UX1 = U12 --> (230, 0) - (492.99999999999994, 110.00000000000001) = (611.156549587252, -49.28989383753886)
lasthoek is -49.28989383753886
  611.156549587252 * 50 * cos(-49.28989383753886 - 20) WARNING, make sure to multiply with -1 if the current enters at -
Pas is -32419.39541711384
  611.156549587252 * 50 * sin(-49.28989383753886 - 20) WARNING, make sure to multiply with -1 if the current enters at -
Qas is -85749.69494473559
QX1 is 73950.0
  230 * 50 * cos(0 - 20) WARNING, make sure to multiply with -1 if the current enters at -
Pnet is -32419.395417113843
  230 * 50 * sin(0 - 20) WARNING, make sure to multiply with -1 if the current enters at -
Qnet is 11799.694944735571
machine is operating -11799.694944735587, therefore capacitive
I2 is 10.628809558039165A


(611.156549587252, -49.28989383753886)

In [201]:
from functools import partial
p = partial(asynchrone, 233, 78, 45, 10)

In [73]:
import altair as alt
import numpy as np
import pandas as pd


source = pd.DataFrame({
  's': s,
  't': t
})

alt.Chart(source).mark_line().encode(
    x='s',
    y='t'
)

NameError: name 't' is not defined

In [ ]:
import numpy as np
s = np.linspace(-0.99,0.99,20)
net = 50


def tas(slip):
    was = 2 * math.pi * net * (1-slip)
    print(slip)
    return p(slip) / was
t = [tas(x) for x in s]

-0.99
converting {var} into carthesian to calculate with
current in polar notation (2.8457072716371075, -72.29599041025817)
converting {var} into polar to calculate with
total reactance (24.8989898989899+78j)
current (2.8457072716371075, -72.29599041025817)
  233 * 2.8457072716371075 * cos(0 - -72.29599041025817) WARNING, make sure to multiply with -1 if the current enters at -
vermogen 604.8997861807899
  233 * 2.8457072716371075 * sin(0 - -72.29599041025817) WARNING, make sure to multiply with -1 if the current enters at -
blindvermogen 1894.9436709485044
asvermogen -488.3369470587314
  233 * 2.8457072716371075 * cos(0 - -72.29599041025817) WARNING, make sure to multiply with -1 if the current enters at -
rendement bij slip > 0 (anders 1/x doen) -0.8073022312373233
-0.8857894736842105
converting {var} into carthesian to calculate with
current in polar notation (2.8580472598977225, -73.09164552846183)
converting {var} into polar to calculate with
total reactance (23.710635769459298+78

0.1
converting {var} into carthesian to calculate with
current in polar notation (26.218383414428498, -13.657642505812152)
converting {var} into polar to calculate with
total reactance (214+52j)
current (26.218383414428498, -13.657642505812152)
  5774 * 26.218383414428498 * cos(0 - -13.657642505812152) WARNING, make sure to multiply with -1 if the current enters at -
vermogen 441313.12973195873
  5774 * 26.218383414428498 * sin(0 - -13.657642505812152) WARNING, make sure to multiply with -1 if the current enters at -
blindvermogen 107234.96610309277
asvermogen 389757.8575670103
  5774 * 26.218383414428498 * cos(0 - -13.657642505812152) WARNING, make sure to multiply with -1 if the current enters at -
rendement bij slip > 0 (anders 1/x doen) 0.8831775700934579
0.18888888888888888
converting {var} into carthesian to calculate with
current in polar notation (45.69087730178656, -24.29827663498353)
converting {var} into polar to calculate with
total reactance (115.1764705882353+52j)
current